# Catanomics - Computer Vision and pre-processing
The goal of this notebook is to read in an image of a catan board without number or settlements, and draw the board (ports not included) over the image

In [1]:
# Import necessary libraries
import cv2
import numpy as np

In [2]:
# Show image function for later use
def showImage(img, name=None):
    if not name:
        cv2.imshow("Image display", img)
    else:
        cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
# Save image function
def saveImage(filename, img, dir):
    # Get full path
    full_path = f"{dir}/{filename}"
    cv2.imwrite(full_path, img)
    print(f"Image saved to {full_path}")

## 1. Read in the image, detect the border, and find the max/min y point of the permiter

In [3]:
# Read in an image
image = cv2.imread('../images/v3/board01.jpg')
# The input imgs are too big, so reduce to 25%
image = cv2.resize(image, (0,0), fx=.25, fy=.25)
showImage(image)
og_img = image.copy()

In [4]:
# Convert the image to hsv
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

In [5]:
# Define color bounds for the perimiter of the board
lower_blue = np.array([100, 150, 0])
upper_blue = np.array([140, 255, 255])

# Create a binary mask where the blue regions are white, and everything else is black
mask = cv2.inRange(hsv_image, lower_blue, upper_blue)
showImage(mask)

In [6]:
# Bitwise AND to keep the blue parts of the image
blue_regions = cv2.bitwise_and(image, image, mask=mask)

In [7]:
# Perform canny edge detection on the blue_regions

# Convert to grayscale
gray_blue_region = cv2.cvtColor(blue_regions, cv2.COLOR_BGR2GRAY)
# Apply Gaussian Blur
blurred_blue_regions = cv2.GaussianBlur(gray_blue_region, (5,5), 0)
# Canny edge detection
edges = cv2.Canny(blurred_blue_regions, 50, 150)
# Show the result
showImage(edges, "Edges in blue regions")

# 729.24435816, 601.65764162
print(edges)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## 2. Perform Hough Transform on the images

In [8]:
# Perform Hough Transform using `HoughLines`
lines = []
thresh = 200
while len(lines) < 10:
    lines = cv2.HoughLines(edges, 1, np.pi / 180, thresh)
    print(f"Trying threshold {thresh} | Got {len(lines)} lines.")
    if len(lines) >= 10:
        break
    else:
        thresh -= 5

print(f"Results: {thresh} threshold | {len(lines)} lines")

Trying threshold 200 | Got 3 lines.
Trying threshold 195 | Got 3 lines.
Trying threshold 190 | Got 3 lines.
Trying threshold 185 | Got 3 lines.
Trying threshold 180 | Got 3 lines.
Trying threshold 175 | Got 3 lines.
Trying threshold 170 | Got 3 lines.
Trying threshold 165 | Got 3 lines.
Trying threshold 160 | Got 3 lines.
Trying threshold 155 | Got 4 lines.
Trying threshold 150 | Got 4 lines.
Trying threshold 145 | Got 4 lines.
Trying threshold 140 | Got 5 lines.
Trying threshold 135 | Got 5 lines.
Trying threshold 130 | Got 5 lines.
Trying threshold 125 | Got 6 lines.
Trying threshold 120 | Got 8 lines.
Trying threshold 115 | Got 11 lines.
Results: 115 threshold | 11 lines


In [9]:
# Draw lines on the image
for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    
showImage(image)
saveImage("perimiter00.jpg", image, "../images/perimiter/v1")

Image saved to ../images/perimiter/v1/perimiter00.jpg


In [10]:
print(lines[0, 0, 0])

-331.0


In [11]:
min_rho_difference = 25
overlapping_indexes = []

for i in range(len(lines)):
    rho, _ = lines[i, 0]
    for j in range(i+1, len(lines)):
        rho2, _ = lines[j, 0]
        if abs(rho - rho2) < min_rho_difference:
            print(f"deleting an overlapping line at index {j}: {lines[j]}")
            overlapping_indexes.append(j)
print(f"Overlapping indexes: {overlapping_indexes}")

deleting an overlapping line at index 7: [[240.          2.4783676]]
deleting an overlapping line at index 8: [[242.          2.4783676]]
deleting an overlapping line at index 7: [[240.          2.4783676]]
deleting an overlapping line at index 8: [[242.          2.4783676]]
deleting an overlapping line at index 6: [[745.          1.3439035]]
deleting an overlapping line at index 9: [[123.          1.6406095]]
deleting an overlapping line at index 10: [[7.0500000e+02 3.3161256e-01]]
deleting an overlapping line at index 8: [[242.          2.4783676]]
Overlapping indexes: [7, 8, 7, 8, 6, 9, 10, 8]


In [12]:
perimiter_lines = []
for i in range(len(lines)):
    if i not in overlapping_indexes:
        perimiter_lines.append(lines[i])
        
print(len(perimiter_lines))

6


In [13]:
# This is a list of lists of tuples which are coordinates for each line
line_coords = []
# Calculate start and end points for the lines
for index, line in enumerate(perimiter_lines):
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    # cv2.line(og_img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    print(f"Coordinates of line {index}: ({x1, y1}) to ({x2, y2})")
    line_coords.append([(x1, y1), (x2, y2)])
line_coords[0]

Coordinates of line 0: ((-213, -1031)) to ((786, 700))
Coordinates of line 1: ((-223, 999)) to ((622, -813))
Coordinates of line 2: ((-823, -619)) to ((435, 934))
Coordinates of line 3: ((-824, 932)) to ((1132, 516))
Coordinates of line 4: ((-1006, 29)) to ((985, 203))
Coordinates of line 5: ((326, 1182)) to ((1010, -696))


[(-213, -1031), (786, 700)]

In [14]:
import math
# Function to calculate the slope given two points
def slope(x1,y1,x2,y2):
    ###finding slope
    if x2!=x1:
        return((y2-y1)/(x2-x1))
    else:
        return 'NA'

# Function to calculate the y-intercept given two points 
def y_intercept(x1,y1,x2,y2):
    # y = mx+b OR b = y-mx
    b = y1 - (slope(x1, y1, x2, y2) * x1)
    return b

def calc_intersection(m1,b1,m2,b2):
    # Create the coefficient matrices
    a = np.array([[-m1, 1], [-m2, 1]])
    b = np.array([b1, b2])
    try:
        solution = np.linalg.solve(a, b)
    except:
        return None

    return solution

# Function that draws the lines in the bounds of the image
def drawLine(image,x1,y1,x2,y2):

    m=slope(x1,y1,x2,y2)
    h,w=image.shape[:2]
    if m!='NA':
        ### here we are essentially extending the line to x=0 and x=width
        ### and calculating the y associated with it
        ##starting point
        px=0
        py=-(x1-0)*m+y1
        ##ending point
        qx=w
        qy=-(x2-w)*m+y2
    else:
    ### if slope is zero, draw a line with x=x1 and y=0 and y=height
        px,py=x1,0
        qx,qy=x1,h
    # Draws a green line
    cv2.line(image, (int(px), int(py)), (int(qx), int(qy)), (0, 255, 0), 2)

In [15]:
test_image = og_img.copy()
# List of lists of tuples of slope and y-intercept of linear lines
line_equations = []
print(line_coords)
for line in line_coords:
    line_equations.append((slope(line[0][0], line[0][1], line[1][0], line[1][1]), y_intercept(line[0][0], line[0][1], line[1][0], line[1][1])))

line_equations

[[(-213, -1031), (786, 700)], [(-223, 999), (622, -813)], [(-823, -619), (435, 934)], [(-824, 932), (1132, 516)], [(-1006, 29), (985, 203)], [(326, 1182), (1010, -696)]]


[(1.7327327327327327, -661.927927927928),
 (-2.1443786982248523, 520.8035502958579),
 (1.2344992050874404, 396.99284578696347),
 (-0.21267893660531698, 756.7525562372189),
 (0.0873932697137117, 116.91762933199396),
 (-2.745614035087719, 2077.0701754385964)]

In [16]:
# intersection = calc_intersection(line_equations[0][0], line_equations[0][1], line_equations[1][0], line_equations[1][1])
# cv2.circle(test_image, (int(intersection[0]), int(intersection[1])), 1, (0, 0, 255), -1)
# showImage(test_image)

perimeter_points = []
image_height, image_width, _ = test_image.shape
for ind, line_one in enumerate(line_equations):
    for line_two in line_equations[ind+1:]:
        perimeter_point = calc_intersection(line_one[0], line_one[1], line_two[0], line_two[1])
        if perimeter_point[0] >= 0 and perimeter_point[0] <= image_width and perimeter_point[1] >= 0 and perimeter_point[1] <= image_height:
            perimeter_points.append((round(perimeter_point[0]), round(perimeter_point[1])))

# for i in range(len(line_equations)):
#     if i+1 == len(line_equations):
#         perimeter_point = calc_intersection(line_equations[i][0], line_equations[i][1], line_equations[0][0], line_equations[0][1])
#     else:
#         perimeter_point = calc_intersection(line_equations[i][0], line_equations[i][1], line_equations[i+1][0], line_equations[i+1][1])
#     perimeter_points.append(perimeter_point)

print(f"Perimeter points: {perimeter_points}")

Perimeter points: [(729, 602), (473, 158), (612, 398), (37, 442), (181, 133), (249, 704), (422, 918), (521, 646), (692, 177)]


### Finding the cloesest edge points with the coordinates found in the intersections (and setting a max distance so we get rid of non-perimeter points)

In [20]:
# Invert the binary image (edges)
inverted_edges = 255 - edges

# Apply distance transform
dist_transform = cv2.distanceTransform(inverted_edges, cv2.DIST_L2, 5)

good_pts = []
# For each point, find the distance to the closest white pixel
for i in range(len(perimeter_points)):
    x, y = perimeter_points[i]
    dist_to_edge = dist_transform[y, x]
    if dist_to_edge < 5:
        good_pts.append(perimeter_points[i])
        # perimeter_points = perimeter_points[0:i] + perimeter_points[i+1:]

for point in good_pts:
    cv2.circle(test_image, (round(point[0]), round(point[1])), 5, (255, 0, 255), -1)
showImage(test_image)

[(729, 602), (473, 158), (612, 398), (37, 442), (181, 133), (249, 704), (422, 918), (521, 646), (692, 177)]
Point (473, 158) has a distance of 0.0 to the closest white pixel
Point (612, 398) has a distance of 1.0 to the closest white pixel
Point (37, 442) has a distance of 0.0 to the closest white pixel
Point (181, 133) has a distance of 2.0 to the closest white pixel
Point (249, 704) has a distance of 1.0 to the closest white pixel
Point (521, 646) has a distance of 0.0 to the closest white pixel
